In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
import os
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.utils import class_weight
from sklearn.preprocessing import minmax_scale

import random
import cv2
from imgaug import augmenters as iaa
import warnings
warnings.filterwarnings('ignore')


import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.experimental import CosineDecay
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import EfficientNetB3, ResNet50
from tensorflow.keras.layers.experimental.preprocessing import RandomCrop,CenterCrop, RandomRotation

In [ ]:
label = pd.read_csv('../input/clothing-dataset-full/images.csv')

In [ ]:
labels = list(label['label'].unique())

In [ ]:
labels

In [ ]:
path = '../input/clothing-dataset-full/images_compressed/'
y = pd.get_dummies(label['label'])
x = pd.DataFrame()
x['image_id'] = label['image'] + '.jpg'
#x['label'] = label['label'] 
x['filepath'] = path+ label['image'] + '.jpg'

In [ ]:
file_list = os.listdir('../input/clothing-dataset-full/images_compressed')

In [ ]:
x = list(x['filepath'].values)

In [ ]:
for idx,tmp in enumerate(x):
    if (tmp in file_list):
        print('False in ' + str(idx))

In [ ]:
path = '../input/clothing-dataset-full/images_compressed/'
y = pd.get_dummies(label['label'])
x = pd.DataFrame()
x['image_id'] = label['image'] + '.jpg'
#x['label'] = label['label'] 
x['filepath'] = path+ label['image'] + '.jpg'

In [ ]:
x = list(x['filepath'].values)
y = list(y.values)



In [ ]:
delete = [533,702,859,1659,1759]
for de in delete:
    del y[de]
    del x[de]
    

In [ ]:
# batch_size = 32
# training_data = tf.data.Dataset.from_tensor_slices((x,y))
# #validation_data = tf.data.Dataset.from_tensor_slices((validation_df.filepath.values, validation_df.label.values))
# def load_image_and_label_from_path(image_path, label):
#     img = tf.io.read_file(image_path)
#     img = tf.image.decode_jpeg(img, channels=3)
#     img = layers.experimental.preprocessing.Rescaling(1.0 / 255)(img)
#     return img, label


# training_data = training_data.map(load_image_and_label_from_path)
# # training_data_batches = training_data.shuffle(buffer_size=1000).batch(batch_size)

In [ ]:
img_size = 224
img_shape = (img_size,img_size,3)
batch_size = 32
epochs = 20
dropout_rate = 0.5
num_of_predict = len(y[0])
len_data = 4000

In [ ]:
train_img = []
for idx,img in enumerate(x):
    try:
        tmp = cv2.imread(img
                                    , cv2.IMREAD_COLOR)
        tmp = cv2.resize(tmp,dsize=(img_size,img_size),
                                interpolation=cv2.INTER_AREA)
        train_img.append(tmp)
    except Exception as e:
        print(str(e)+str(idx))
train_img = np.array(train_img)

# delete = [533,702,859,1659,1759]
# for de in delete:
#     del y[de]

In [ ]:
# delete = [533,702,859,1659,1759]
# for de in delete:
#     del y[de]

In [ ]:
print(len(train_img),len(y))

In [ ]:
train_img = np.array(train_img)/255.
y = np.array(y)

In [ ]:
train_x = train_img[:4000]
val_x = train_img[4000:]
train_y = y[:4000]
val_y = y[4000:]

In [ ]:
del train_img
del y

In [ ]:
# ResNet50 = ResNet50(
#         include_top=False, weights='imagenet',
#         input_shape=img_shape, classes=13
#     )

# inputs = Input(shape=img_shape)
# ResNet50 = ResNet50(inputs) 
# pooling = layers.GlobalAveragePooling2D()(ResNet50) 
# dropout = layers.Dropout(dropout_rate)(pooling) #dropout
# outputs = Dense(len(y[0]), activation="softmax")(dropout)
# model = Model(inputs=inputs, outputs=outputs)

In [ ]:
data_augmentation_layers = tf.keras.Sequential(
    [
        layers.experimental.preprocessing.RandomCrop(height=img_size, width=img_size),
        layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        layers.experimental.preprocessing.RandomRotation(0.25),
        layers.experimental.preprocessing.RandomZoom((-0.2, 0)),
        layers.experimental.preprocessing.RandomContrast((0.2,0.2))
    ]
)

In [ ]:
efficientnet = EfficientNetB3(weights="../input/keras-efficientnetb3-no-top-weights/efficientnetb3_notop.h5",
                              include_top=False, 
                              input_shape=img_shape, 
                              drop_connect_rate=dropout_rate)

inputs = Input(shape=img_shape)
#augmented = data_augmentation_layers(inputs) #Augmentation
efficientnet = efficientnet(inputs) #efficientnet
pooling = layers.GlobalAveragePooling2D()(efficientnet) #globalaveragepooling
dropout = layers.Dropout(dropout_rate)(pooling) #dropout
outputs = Dense(len(val_y[0]), activation="softmax")(dropout)
model2 = Model(inputs=inputs, outputs=outputs)

In [ ]:
decay_steps = int(round(4000*0.8/batch_size))*epochs
cosine_decay = CosineDecay(initial_learning_rate=3e-5, decay_steps=decay_steps, alpha=0.3)

model2.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(cosine_decay), metrics=["accuracy"])

In [ ]:
model2.summary()

In [ ]:
# history = model2.fit(training_data,batch_size=32,
#                   epochs = 1 )

In [ ]:
history = model2.fit(train_x,train_y, batch_size=batch_size,
                   epochs = 25,
                   validation_data=(val_x, val_y))

In [ ]:
model2.evaluate(x=val_x,y= val_y)

In [ ]:
model2.save('eff_final.h5')

from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

image = Image.open(tmp.iloc[0]['filepath'])
plt.imshow(image)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
tt =list(range(len(history.history['loss'])))
plt.plot(tt,history.history['loss'])
plt.plot(tt,history.history['val_loss'])
plt.title('loss')
plt.show

In [ ]:
plt.plot(tt,history.history['accuracy'])
plt.plot(tt,history.history['val_accuracy'])
plt.title('accuracy')
plt.show

In [ ]:
import cv2
in_path = '../input/cloth-test/testing2.png'

img = cv2.imread(in_path)
img = cv2.resize(img, (img_size, img_size))  # resize image to match model's expected sizing
img = np.reshape(img, [1, img_size, img_size, 3])
img = img / 255.
out = model.predict(img)
# out = np.argmax(out)
# labels[out]

In [ ]:
# def predict(model, in_path):
#     img = cv2.imread(in_path)
#     img = cv2.resize(img, (img_size, img_size))  # resize image to match model's expected sizing
#     img = np.reshape(img, [1, img_size, img_size, 3])
#     img = img / 255.
#     out = model.predict(img)
#     return labels[np.argmax(out)]


In [ ]:
# import cv2
# in_path = '../input/cloth-test/testing2.png'
# predict(model,in_path)

In [ ]:
# img = cv2.imread(in_path)
# plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
# plt.title(predict(model,in_path))

# plt.show()

In [ ]:
labels

# Get Color

In [ ]:
# in_path= '../input/clothing-dataset-full/images_compressed/8f6822c3-862b-4126-9a35-585569aec240.jpg'
# img = cv2.imread(in_path)
# plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

dddddddddddddddddddddddddddd

In [ ]:
# label[label['label']=='Other']

In [ ]:
# labels
# rows = 4
# cols = 5
# fig = plt.figure()
# for x in labels:
#     tmp = label[label['label']==x].iloc[0]
#     tmp = tmp['image']
#     in_path= '../input/clothing-dataset-full/images_compressed/'+tmp+'.jpg'
#     img = cv2.imread(in_path)
#     plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
# fig = plt.figure(figsize=(12, 12)) # rows*cols 행렬의 i번째 subplot 생성
# rows = 4
# cols = 5
# i = 1
 
# #xlabels = ["xlabel", "(a)", "(b)", "(c)", "(d)"]
# xlabels = ['xlabel']+labels 
# for x in labels:
#     tmp = label[label['label']==x].iloc[1]
#     tmp = tmp['image']
#     in_path= '../input/clothing-dataset-full/images_compressed/'+tmp+'.jpg'
#     img = cv2.imread(in_path)
#     ax = fig.add_subplot(rows, cols, i)
#     ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
#     ax.set_xlabel("%s6%s"%(xlabels[i],return_color(img)))
#     ax.set_xticks([]), ax.set_yticks([])
#     i += 1
 
# plt.show()


In [ ]:
# labels

tensorflow			무신사	
'Not sure',				
 'T-Shirt',	반팔	상의	반팔티셔츠	
 'Shoes',	신발			
 'Shorts',	반바지	바지	숏 팬츠	
 'Shirt',	셔츠	상의	셔츠/블라우스	
 'Pants',	긴바지	바지	데님 팬츠, 코튼 팬츠, 레깅스, 슈트 팬츠/슬랙스, 트레이닝/조거 팬츠, 기타 바지	
 'Skirt',	치마	스커트	스커트 다	
 'Other',				
 'Top',	나시? 탑?	상의	민소매티셔츠	
 'Outwear',	아우터,겉옷	아우터	아우터에서 슈트 블레이저 빼고 다	
 'Dress',	드레스	원피스 	원피스 다	?
 'Body',	한벌옷?	바지	점프슈트/오버올	
 'Longsleeve',	긴팔,맨투맨	상의	긴팔 티셔츠,맨투맨/스웨트셔츠,니트/스웨터	
 'Undershirt',				
 'Hat',	모자			
 'Polo',	폴로티?	상의	피케/카라티셔츠	
 'Blouse',	블라우져	상의	셔츠/블라우스	
 'Hoodie',	후드티	상의	후드티셔츠	
 'Skip',	원피스	원피스 	원피스 다	?
 'Blazer'	블레이저	아우터	슈트/블레이저	\

해야할일  
색 ~~~  
추천 알고리즘  (명확히)

In [ ]:
# import math
# from collections import Counter

# def find_nearest(array,value):
#     idx = np.searchsorted(array, value, side="left")
#     if idx > 0 and (idx == len(array) or math.fabs(value - array[idx-1]) < math.fabs(value - array[idx])):
#         return array[idx-1]
#     else:
#         return array[idx]

# def get_color_distance(a,b):
#     tmp = 0
#     for x in range(3):
#         tmp += (a[x]-b[x])**2
#     return math.sqrt(tmp)


# def split_map(x):
#     return list(map(int,x.split(',')))

# color_table = pd.read_csv('../input/color-table/color_table (2).csv')
# table_list = list(color_table['rgb'].values)
# table_list = list(map(split_map,table_list))
# table_col = list(color_table['eng'].values)
# def return_color(img):
#     color=[]
#     color.append(img[int(img.shape[0]/2)][int(img.shape[1]/2)].tolist()[::-1])
#     color.append(img[int(img.shape[0]/4)][int(img.shape[1]/4)].tolist()[::-1])
#     color.append(img[int(img.shape[0]*3/4)][int(img.shape[1]/4)].tolist()[::-1])
#     color.append(img[int(img.shape[0]/4)][int(img.shape[1]*3/4)].tolist()[::-1])
#     color.append(img[int(img.shape[0]*3/4)][int(img.shape[1]*3/4)].tolist()[::-1])
    

#     color_dis = []
#     all_color=[]
#     for col in color:
#         color_dis = []
#         for x in table_list:
#             color_dis.append(get_color_distance(col,x))
#         all_color.append(table_col[np.argmin(color_dis)])
#     cnt = Counter(all_color)
#     return list(cnt)[0]
#     return table_col[np.argmin(color_dis)]

In [ ]:
# in_path= '../input/clothing-dataset-full/images_compressed/8f6822c3-862b-4126-9a35-585569aec240.jpg'
# img = cv2.imread(in_path)